In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [6]:
df = pd.read_csv(r'D:\python_project\chaekchecklab\data\emb_value.csv', index_col=0)
# df

In [4]:
from scipy.sparse import load_npz

# csr_matrix 불러오기
tfidf_matrix = load_npz(r'D:\python_project\chaekchecklab\data\tfidf_matrix.npz')

# print(tfidf_matrix)

In [5]:
def get_related_books(q_book, df, tfidf_matrix, k1=2000, k2=10, alpha=0.5):
    # $$$ 여기서 예외처리 필요
    q_idx = df[df['title'] == q_book].index[0]
    cosine_sim_1 = cosine_similarity(tfidf_matrix[q_idx], tfidf_matrix).flatten()

    # 유사도 top_k1 인덱스들
    # similar_book_idxes = cosine_sim_1.argsort()[-(k1+1):][::-1]
    similar_k1_idxes = cosine_sim_1.argpartition(-(k1+1))[-(k1+1):]
    df_top_k1 = df.loc[similar_k1_idxes]
    # df_top_k1['similarity'] = cosine_sim_1[similar_book_idxes]
    tfidf_top_k1 = tfidf_matrix[similar_k1_idxes, :].toarray()

    # MMR
    cosine_sim_2 = cosine_similarity(tfidf_top_k1, tfidf_top_k1)

    # 쿼리 책 & 유사도
    np_idx = np.argwhere(similar_k1_idxes==q_idx).flatten()[0]
    sim_scores = cosine_sim_2[np_idx]

    # MMR을 위한 초기 후보 목록 생성
    recommended_indexes = []
    ranked_indices = np.argsort(sim_scores)[::-1]  # 내림차순 정렬

    # MMR Search
    for i in ranked_indices[1:]:
        if len(recommended_indexes) >= k2:
            break
        
        if i not in recommended_indexes:
            # MMR 계산
            relevance_score = sim_scores[i]
            diversity_score = sum(cosine_sim_2[i][j] for j in recommended_indexes) if recommended_indexes else 0
            mmr_score = alpha * relevance_score - (1 - alpha) * diversity_score

            # print(i, mmr_score, relevance_score, diversity_score)
            # MMR 점수가 높은 항목 선택
            if mmr_score > 0:
                # print(i, df_top_k1.iloc[i, 0])
                recommended_indexes.append(i)
    
    # 추천 결과 반환
    recommended_books = df_top_k1.iloc[recommended_indexes].copy()
    recommended_books['similarity'] = [sim_scores[i] for i in recommended_indexes]
    return recommended_books[['title', 'similarity']]

q_book = "회계 천재가 된 홍대리"
res = get_related_books(q_book, df, tfidf_matrix)
res

,title,similarity
18619,회계 천재가 된 홍대리 1,0.744712
49304,세상에서 가장 쉬운 회계책,0.386363
45590,굿 타이밍,0.152103
33998,사람을 얻는 질문법 38,0.117868
28718,40대 다시 한 번 도전에 미쳐라,0.100085
45416,돈 잘 버는 회사의 명쾌한 절세비법,0.075945
44992,기사되는 보도자료 만들기,0.069178


책 여러 권 선택

In [8]:
import math

In [17]:
q_book_list = ['돈의 물리학', '회계 천재가 된 홍대리', '트렌드 코리아 2025']
k = 10
n = len(q_book_list)
bpk = math.ceil(k/n)

results = []
for q_book in q_book_list:
    print(f"'{q_book}'과 비슷한 책 찾는 중")
    result = get_related_books(q_book, df, tfidf_matrix)[:bpk]
    results.append(result)

df_result = pd.concat(results).iloc[:k, :]

'돈의 물리학'과 비슷한 책 찾는 중
'회계 천재가 된 홍대리'과 비슷한 책 찾는 중
'트렌드 코리아 2025'과 비슷한 책 찾는 중


In [18]:
df_result

,title,similarity
6075,퀀트의 세계,0.488969
5745,절대적이며 상대적인 리더십의 물리학,0.210316
2469,헤지펀드 열전,0.203786
18883,"예측, 일단 의심하라",0.193973
18619,회계 천재가 된 홍대리 1,0.744712
49304,세상에서 가장 쉬운 회계책,0.386363
45590,굿 타이밍,0.152103
33998,사람을 얻는 질문법 38,0.117868
32291,희망 PLUS 멘토 열풍,0.220040
35516,스마트하지 않은 스마트 전쟁,0.205924


### 이전 작업 기록

In [3]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['책소개_토큰'])

In [10]:
from scipy.sparse import save_npz

# csr_matrix(sparse_matrix) 저장
save_npz('tfidf_matrix.npz', tfidf_matrix)

유사도 랭킹 후 1차 컷

In [99]:
# 쿼리 책에 대하여, 모든 책들과의 코사인 유사도 계산
q_book = "회계 천재가 된 홍대리"
k1 = 2000

q_idx = df[df['title'] == q_book].index[0]
cosine_sim_1 = cosine_similarity(tfidf_matrix[q_idx], tfidf_matrix).flatten()
# similar_book_idxes = cosine_sim_1.argsort()[-(k1+1):][::-1]
similar_book_idxes = cosine_sim_1.argpartition(-(k1+1))[-(k1+1):]
similar_book_idxes

array([42498,  5876, 42324, ..., 49946, 49944, 52112], dtype=int64)

In [100]:
# 추천 결과와 유사도 점수를 함께 반환
df_top_k1 = df.iloc[similar_book_idxes].copy()
df_top_k1['similarity'] = cosine_sim_1[similar_book_idxes]
df_top_k1.head(2)

,title,full_name,book_url,publisher,author_list,intro,category,date,pages,isbn,저자,역자,기타,책소개_토큰,embedding_책소개,similarity
42498,마케팅코드,마케팅코드 소설로 읽는 마케팅 이야기,https://www.yes24.com/Product/Goods/3001590,21세기북스,"[['스티븐 브라운', '119220']]",포스트모던 마케팅으로 마케팅에 새로운 패러다임을 제시했던 괴짜 마케팅학자 스티븐 브...,1001025009001,2008-06-25,583쪽,9788935211012,스티븐 브라운,NaN,NaN,"['포스트', '모던', '마케팅', '마케팅', '새로운', '패러다임', '제시...",[ 0.11396833 0.5989598 -0.03949952 -0.302230...,0.059438
5876,"리더의 시, 리더의 격","리더의 시, 리더의 격 탁월한 리더를 위한 인문 경영 바이블 [ 양장 ]",https://www.yes24.com/Product/Goods/114913747,한국경제신문사(한경비피),"[['고두현', '94558'], ['황태인', '405862']]",시에서 발견한 삶의 지혜 경영에서 깨달은 일의 품격감성적인 시와 스토리에 의 경영 ...,1001025001001,2022-10-28,312쪽,9788965961932,"고두현,황태인",NaN,NaN,"['시', '발견', '한', '삶', '지혜', '경영', '깨달', '일', '...",[-0.27416667 0.6174609 0.24831763 0.066118...,0.059455


MMR

In [102]:
tfidf_top_k1 = tfidf_matrix[similar_book_idxes, :].toarray()

cosine_sim_2 = cosine_similarity(tfidf_top_k1, tfidf_top_k1)

In [124]:
k2 = 10
alpha = 0.5

# 쿼리 책 & 그에 대한 유사도
idx = np.argwhere(similar_book_idxes==q_idx).flatten()[0]
# 쿼리 책에 대한 유사도
scores = cosine_sim_2[idx]

# MMR을 위한 초기 후보 목록 생성
recommended_indexes = []
ranked_indices = np.argsort(scores)[::-1]  # 내림차순 정렬

# MMR
for i in ranked_indices[1:]:
    if len(recommended_indexes) >= k2:
        break
    
    if i not in recommended_indexes:
        # MMR 계산
        relevance_score = scores[i]
        diversity_score = sum(cosine_sim_2[i][j] for j in recommended_indexes) if recommended_indexes else 0
        mmr_score = alpha * relevance_score - (1 - alpha) * diversity_score

        # print(i, mmr_score, relevance_score, diversity_score)
        # MMR 점수가 높은 항목 선택
        if mmr_score > 0:
            # print(i, df_top_k1.iloc[i, 0])
            recommended_indexes.append(i)

# 추천 결과 반환
recommended_books = df_top_k1.iloc[recommended_indexes].copy()
recommended_books['similarity'] = [scores[i] for i in recommended_indexes]

In [128]:
recommended_books[['title', 'similarity']]

,title,similarity
18619,회계 천재가 된 홍대리 1,0.744712
49304,세상에서 가장 쉬운 회계책,0.386363
45590,굿 타이밍,0.152103
33998,사람을 얻는 질문법 38,0.117868
28718,40대 다시 한 번 도전에 미쳐라,0.100085
45416,돈 잘 버는 회사의 명쾌한 절세비법,0.075945
44992,기사되는 보도자료 만들기,0.069178
